In [ ]:
import numpy as np
import pandas as pd
import random
from itertools import product
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score

import pygad
import warnings
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings("ignore", category=ConvergenceWarning)


def generate_feature(feature_type: str, length: int) -> np.ndarray:
    if feature_type == "binary":
        return np.random.choice([0, 1], size=length)
    elif feature_type == "nominal":
        return np.random.choice(['A', 'B', 'C', 'D'], size=length)
    elif feature_type == "ordinal":
        return np.random.choice([1, 2, 3, 4, 5], size=length)
    elif feature_type == "quantitative":
        return np.random.uniform(0, 100, size=length)
    else:
        raise ValueError("Неизвестный тип признака")


def create_dataset(num_features: int, num_samples: int) -> pd.DataFrame:
    feature_types = ["binary", "nominal", "ordinal", "quantitative"]
    chosen_types = random.sample(feature_types, k=min(4, num_features))
    while len(chosen_types) < num_features:
        chosen_types.append(random.choice(feature_types))
    random.shuffle(chosen_types)

    data = {}
    for i in range(num_features):
        data[f"Obj1_Feat{i+1}"] = generate_feature(
            chosen_types[i], num_samples)
    for i in range(num_features):
        data[f"Obj2_Feat{i+1}"] = generate_feature(
            chosen_types[i], num_samples)

    labels = []
    for idx in range(num_samples):
        matches = sum(
            data[f"Obj1_Feat{f}"][idx] == data[f"Obj2_Feat{f}"][idx]
            for f in range(1, num_features+1)
        )
        labels.append("Yes" if matches >= num_features // 2 else "No")

    data["Collision"] = labels
    return pd.DataFrame(data)


def fitness_func(ga_instance, solution, solution_idx, model_class, X_train, y_train, X_val, y_val):
    if model_class == LogisticRegression:
        params = {
            'C': solution[0],
            'max_iter': int(solution[1])
        }
        model = LogisticRegression(**params, solver='lbfgs')
    elif model_class == DecisionTreeClassifier:
        params = {
            'max_depth': int(solution[0]),
            'min_samples_split': int(solution[1])
        }
        model = DecisionTreeClassifier(**params)
    else:
        return 0

    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    return f1_score(y_val, preds)


def optimize(model_class, X_train, y_train, X_val, y_val, param_bounds, generations=20, parents_mating=5):
    num_params = len(param_bounds)

    def fitness_wrapper(ga_instance, solution, solution_idx):
        return fitness_func(ga_instance, solution, solution_idx, model_class, X_train, y_train, X_val, y_val)

    ga_instance = pygad.GA(
        num_generations=generations,
        num_parents_mating=parents_mating,
        fitness_func=fitness_wrapper,
        sol_per_pop=10,
        num_genes=num_params,
        gene_space=[{'low': low, 'high': high}
                    for (low, high) in param_bounds],
        mutation_type="random",
        mutation_num_genes=1
    )

    ga_instance.run()
    best_solution, best_fitness, _ = ga_instance.best_solution()
    print(f"Оптимальные параметры: {best_solution}, F1: {best_fitness:.4f}")
    return best_solution, best_fitness


if __name__ == "__main__":
    num_features = 6
    num_samples = 200
    data = create_dataset(num_features, num_samples)

    print(data.head())

    X = data.drop(columns=["Collision"])
    y = data["Collision"].map({"Yes": 1, "No": 0})

    for col in X.columns:
        if X[col].dtype == object:
            X[col] = LabelEncoder().fit_transform(X[col])

    X_train_full, X_test, y_train_full, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_full, y_train_full, test_size=0.25, random_state=42)

    logreg_bounds = [(0.001, 10.0), (100, 1000)]
    dtree_bounds = [(1, 20), (2, 20)]

    best_logreg_params, best_logreg_score = optimize(
        LogisticRegression,
        X_train, y_train,
        X_val, y_val,
        logreg_bounds,
        generations=30,
        parents_mating=5
    )

    best_dtree_params, best_dtree_score = optimize(
        DecisionTreeClassifier,
        X_train, y_train,
        X_val, y_val,
        dtree_bounds,
        generations=30,
        parents_mating=5
    )

    final_logreg = LogisticRegression(
        C=best_logreg_params[0],
        max_iter=int(best_logreg_params[1]),
        solver='lbfgs'
    )
    final_logreg.fit(X_train_full, y_train_full)

    final_dtree = DecisionTreeClassifier(
        max_depth=int(best_dtree_params[0]),
        min_samples_split=int(best_dtree_params[1])
    )
    final_dtree.fit(X_train_full, y_train_full)

    save_dir = Path("saved_models")
    save_dir.mkdir(exist_ok=True)

    import pickle

    with open(save_dir / "best_logreg_model.pkl", "wb") as f:
        pickle.dump(final_logreg, f)

    with open(save_dir / "best_dtree_model.pkl", "wb") as f:
        pickle.dump(final_dtree, f)

    print("Модели сохранены в папке 'saved_models'.")

   Obj1_Feat1  Obj1_Feat2 Obj1_Feat3  Obj1_Feat4  Obj1_Feat5  Obj1_Feat6  \
0           4           1          B           2           1   22.747254   
1           5           1          C           2           1   14.403825   
2           5           0          B           4           1   31.233283   
3           1           1          A           1           0   98.144330   
4           2           1          B           2           0   50.550273   

   Obj2_Feat1  Obj2_Feat2 Obj2_Feat3  Obj2_Feat4  Obj2_Feat5  Obj2_Feat6  \
0           4           0          B           4           1   20.815039   
1           5           0          A           5           0   61.999992   
2           1           0          C           1           1    9.389026   
3           2           0          A           4           0   90.512282   
4           5           0          B           2           1   80.411413   

  Collision  
0       Yes  
1        No  
2        No  
3        No  
4        No  
Оп

In [ ]:
import numpy as np
import pandas as pd
import random
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score

import pyswarms as ps
import pickle


def generate_feature(feature_type: str, length: int) -> np.ndarray:
    if feature_type == "binary":
        return np.random.choice([0, 1], size=length)
    elif feature_type == "nominal":
        return np.random.choice(['A', 'B', 'C', 'D'], size=length)
    elif feature_type == "ordinal":
        return np.random.choice([1, 2, 3, 4, 5], size=length)
    elif feature_type == "quantitative":
        return np.random.uniform(0, 100, size=length)
    else:
        raise ValueError("Неизвестный тип признака")


def create_dataset(num_features: int, num_samples: int) -> pd.DataFrame:
    feature_types = ["binary", "nominal", "ordinal", "quantitative"]
    chosen_types = random.sample(feature_types, k=min(4, num_features))

    while len(chosen_types) < num_features:
        chosen_types.append(random.choice(feature_types))
    random.shuffle(chosen_types)

    data = {}
    for i in range(num_features):
        data[f"Obj1_Feat{i + 1}"] = generate_feature(
            chosen_types[i], num_samples)
    for i in range(num_features):
        data[f"Obj2_Feat{i + 1}"] = generate_feature(
            chosen_types[i], num_samples)

    labels = []
    for idx in range(num_samples):
        matches = sum(
            data[f"Obj1_Feat{f}"][idx] == data[f"Obj2_Feat{f}"][idx]
            for f in range(1, num_features + 1)
        )
        labels.append("Yes" if matches >= num_features // 2 else "No")
    data["Collision"] = labels
    return pd.DataFrame(data)


def pso_fitness(params, model_class, X_train, y_train, X_val, y_val):
    n_particles = params.shape[0]
    fitness = np.zeros(n_particles)

    for i in range(n_particles):
        p = params[i]
        if model_class == LogisticRegression:
            C = p[0]
            max_iter = int(p[1])
            max_iter = max(max_iter, 1000)
            model = LogisticRegression(C=C, max_iter=max_iter, solver='lbfgs')
        elif model_class == DecisionTreeClassifier:
            max_depth = int(p[0])
            min_samples_split = int(p[1])
            model = DecisionTreeClassifier(
                max_depth=max_depth, min_samples_split=min_samples_split)
        else:
            fitness[i] = 1.0
            continue

        if len(np.unique(y_train)) < 2:
            fitness[i] = 1.0
            continue

        try:
            model.fit(X_train, y_train)
            preds = model.predict(X_val)
            score = f1_score(y_val, preds)
            fitness[i] = -score
        except Exception:
            fitness[i] = 1.0

    return fitness


def optimize_pso(model_class, X_train, y_train, X_val, y_val, bounds, options, iters=50):
    optimizer = ps.single.GlobalBestPSO(
        n_particles=20,
        dimensions=len(bounds[0]),
        options=options,
        bounds=bounds
    )

    best_cost, best_pos = optimizer.optimize(
        pso_fitness, iters,
        model_class=model_class, X_train=X_train, y_train=y_train,
        X_val=X_val, y_val=y_val
    )

    print(f"Лучшие параметры (PSO): {best_pos}, F1: {-best_cost:.4f}")
    return best_pos, -best_cost


if __name__ == "__main__":
    num_features = 6
    num_samples = 200

    data = create_dataset(num_features, num_samples)
    print(data.head())

    X = data.drop(columns=["Collision"])
    y = data["Collision"].map({"Yes": 1, "No": 0})

    for col in X.columns:
        if X[col].dtype == object:
            X[col] = LabelEncoder().fit_transform(X[col])

    quant_cols = [col for col in X.columns if X[col].dtype in [
        np.float64, np.float32]]
    scaler = StandardScaler()
    if quant_cols:
        X[quant_cols] = scaler.fit_transform(X[quant_cols])

    X_train_full, X_test, y_train_full, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_full, y_train_full, test_size=0.25, random_state=42)

    print("Распределение классов в полном тренировочном наборе:",
          np.unique(y_train_full, return_counts=True))
    print("Распределение классов в тренировочном наборе:",
          np.unique(y_train, return_counts=True))

    logreg_bounds = (np.array([0.001, 100]), np.array([10.0, 1500]))
    dtree_bounds = (np.array([1, 2]), np.array([20, 20]))

    options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}

    best_logreg_params, best_logreg_score = optimize_pso(
        LogisticRegression, X_train, y_train, X_val, y_val,
        bounds=logreg_bounds, options=options, iters=50
    )

    best_dtree_params, best_dtree_score = optimize_pso(
        DecisionTreeClassifier, X_train, y_train, X_val, y_val,
        bounds=dtree_bounds, options=options, iters=50
    )

    print("Распределение классов перед финальным обучением:",
          np.unique(y_train_full, return_counts=True))
    if len(np.unique(y_train_full)) < 2:
        raise ValueError(
            "Ошибка: в полном тренировочном наборе только один класс!")

    final_logreg = LogisticRegression(
        C=best_logreg_params[0],  # Оптимальный C
        max_iter=int(max(best_logreg_params[1], 1000)),
        solver='lbfgs'
    )
    final_logreg.fit(X_train_full, y_train_full)

    final_dtree = DecisionTreeClassifier(
        max_depth=int(best_dtree_params[0]),
        min_samples_split=int(best_dtree_params[1])
    )
    final_dtree.fit(X_train_full, y_train_full)

    save_dir = Path("saved_models")
    save_dir.mkdir(exist_ok=True)

    with open(save_dir / "best_logreg_pso_model.pkl", "wb") as f:
        pickle.dump(final_logreg, f)

    with open(save_dir / "best_dtree_pso_model.pkl", "wb") as f:
        pickle.dump(final_dtree, f)

    print("Модели успешно сохранены в 'saved_models'.")

2025-06-17 09:02:14,669 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}


   Obj1_Feat1  Obj1_Feat2  Obj1_Feat3 Obj1_Feat4  Obj1_Feat5  Obj1_Feat6  \
0           0           1   94.038430          C           0           0   
1           0           1   85.672510          A           0           1   
2           1           4   81.876258          B           1           1   
3           0           1   31.202701          B           0           1   
4           0           4   49.163069          A           1           0   

   Obj2_Feat1  Obj2_Feat2  Obj2_Feat3 Obj2_Feat4  Obj2_Feat5  Obj2_Feat6  \
0           1           3   88.536080          B           1           0   
1           0           4   37.242611          D           1           1   
2           0           1   24.295041          B           1           1   
3           0           2   13.142731          C           0           0   
4           0           5    2.018540          D           0           0   

  Collision  
0        No  
1        No  
2       Yes  
3        No  
4        No  
Ра

pyswarms.single.global_best: 100%|██████████|50/50, best_cost=-0
2025-06-17 09:02:22,856 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -0.0, best pos: [  9.8316126  312.78439627]
2025-06-17 09:02:22,866 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}


Лучшие параметры (PSO): [  9.8316126  312.78439627], F1: 0.0000


pyswarms.single.global_best: 100%|██████████|50/50, best_cost=-0.308
2025-06-17 09:02:25,964 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -0.3076923076923077, best pos: [ 3.97948831 14.16170483]


Лучшие параметры (PSO): [ 3.97948831 14.16170483], F1: 0.3077
Распределение классов перед финальным обучением: (array([0, 1]), array([115,  45]))
Модели успешно сохранены в 'saved_models'.


In [ ]:
import neat
import numpy as np
import pandas as pd
import random
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from pathlib import Path
import warnings

warnings.filterwarnings("ignore")


def generate_feature(feature_type: str, length: int):
    if feature_type == "binary":
        return np.random.choice([0, 1], size=length)
    elif feature_type == "nominal":
        return np.random.choice(['A', 'B', 'C', 'D'], size=length)
    elif feature_type == "ordinal":
        return np.random.choice([1, 2, 3, 4, 5], size=length)
    elif feature_type == "quantitative":
        return np.random.uniform(0, 100, size=length)
    else:
        raise ValueError("Unknown feature type")


def create_dataset(num_features: int, num_samples: int) -> pd.DataFrame:
    feature_types = ["binary", "nominal", "ordinal", "quantitative"]
    chosen_types = random.sample(feature_types, k=min(4, num_features))
    while len(chosen_types) < num_features:
        chosen_types.append(random.choice(feature_types))
    random.shuffle(chosen_types)

    data = {}
    for i in range(num_features):
        data[f"Obj1_Feat{i + 1}"] = generate_feature(
            chosen_types[i], num_samples)
    for i in range(num_features):
        data[f"Obj2_Feat{i + 1}"] = generate_feature(
            chosen_types[i], num_samples)

    labels = []
    for idx in range(num_samples):
        matches = sum(
            data[f"Obj1_Feat{f}"][idx] == data[f"Obj2_Feat{f}"][idx]
            for f in range(1, num_features + 1)
        )
        labels.append("Yes" if matches >= num_features // 2 else "No")
    data["Collision"] = labels
    return pd.DataFrame(data)


def prepare_dataset(num_features, num_samples):
    df = create_dataset(num_features, num_samples)
    X = df.drop(columns=["Collision"])
    y = df["Collision"].map({"Yes": 1, "No": 0})

    for col in X.columns:
        if X[col].dtype == object:
            X[col] = LabelEncoder().fit_transform(X[col])
    return X.values, y.values


def eval_genome(genome, config, X_train, y_train, X_val, y_val):
    net = neat.nn.FeedForwardNetwork.create(genome, config)
    predictions = []
    for xi in X_val:
        output = net.activate(xi)
        predictions.append(1 if output[0] > 0.5 else 0)
    return f1_score(y_val, predictions)


def run_neat(X_train, y_train, X_val, y_val, config_file, generations=50):
    print(f"Размер данных: {X_train.shape[1]} признаков")

    config = neat.Config(
        neat.DefaultGenome,
        neat.DefaultReproduction,
        neat.DefaultSpeciesSet,
        neat.DefaultStagnation,
        config_file
    )

    population = neat.Population(config)

    population.add_reporter(neat.StdOutReporter(True))
    population.add_reporter(neat.StatisticsReporter())

    def eval_genomes(genomes, config):
        for genome_id, genome in genomes:
            genome.fitness = eval_genome(
                genome, config, X_train, y_train, X_val, y_val)

    winner = population.run(eval_genomes, generations)
    return winner, config


def save_neat_model(winner, config, filename):
    with open(filename, "wb") as f:
        pickle.dump((winner, config), f)


if __name__ == "__main__":
    X_small, y_small = prepare_dataset(num_features=4, num_samples=30)
    X_small = np.hstack([X_small[:, :4], X_small[:, 4:8]])

    X_big, y_big = prepare_dataset(num_features=15, num_samples=1500)

    X_train_s, X_val_s, y_train_s, y_val_s = train_test_split(
        X_small, y_small, test_size=0.2, random_state=42)
    X_train_b, X_val_b, y_train_b, y_val_b = train_test_split(
        X_big, y_big, test_size=0.2, random_state=42)

    config_small_path = "config_small"
    config_big_path = "config_big"

    winner_small, config_small = run_neat(
        X_train_s, y_train_s, X_val_s, y_val_s, config_small_path)
    print("NEAT для малого датасета завершён.")

    winner_big, config_big = run_neat(
        X_train_b, y_train_b, X_val_b, y_val_b, config_big_path)
    print("NEAT для большого датасета завершён.")

    save_dir = Path("saved_models")
    save_dir.mkdir(exist_ok=True)

    save_neat_model(winner_small, config_small, save_dir / "neat_small.pkl")
    save_neat_model(winner_big, config_big, save_dir / "neat_big.pkl")

    print("Модели NEAT сохранены.")

Размер данных: 8 признаков

 ****** Running generation 0 ****** 

Population's average fitness: 0.22610 stdev: 0.20728
Best fitness: 1.00000 - size: (1, 8) - species 1 - id 46

Best individual in generation 0 meets fitness threshold - complexity: (1, 8)
NEAT для малого датасета завершён.
Размер данных: 30 признаков

 ****** Running generation 0 ****** 

Population's average fitness: 0.00000 stdev: 0.00000
Best fitness: 0.00000 - size: (1, 30) - species 1 - id 1
Average adjusted fitness: 0.000
Mean genetic distance 0.955, standard deviation 0.371
Population of 50 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0    50      0.0    0.000     0
Total extinctions: 0
Generation time: 0.183 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.00000 stdev: 0.00000
Best fitness: 0.00000 - size: (1, 30) - species 1 - id 1
Average adjusted fitness: 0.000
Mean genetic distance 0.871, standard deviation 0.293


In [ ]:
import numpy as np
import random
import pickle
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder

from pureples.shared.substrate import Substrate
from pureples.shared.create_cppn import create_cppn
from pureples.es_hyperneat.es_hyperneat import ESNetwork
import neat


class DefaultESParameters:
    def __init__(self):
        self.initial_depth = 1
        self.max_depth = 3
        self.variance_threshold = 0.03
        self.band_threshold = 0.3
        self.iteration_level = 1
        self.division_threshold = 0.5
        self.max_weight = 5.0
        self.activation = "sigmoid"


def preprocess_dataset(df):
    df_copy = df.copy()
    for col in df_copy.columns:
        if df_copy[col].dtype == object:
            df_copy[col] = LabelEncoder().fit_transform(df_copy[col])
    return df_copy


def generate_feature(feature_type, length):
    if feature_type == "binary":
        return np.random.choice([0, 1], size=length)
    elif feature_type == "nominal":
        return np.random.choice(['A', 'B', 'C', 'D'], size=length)
    elif feature_type == "ordinal":
        return np.random.choice([1, 2, 3, 4, 5], size=length)
    elif feature_type == "quantitative":
        return np.random.uniform(0, 100, size=length)
    else:
        raise ValueError("Unknown feature type")


def create_dataset(num_features, num_samples):
    feature_types = ["binary", "nominal", "ordinal", "quantitative"]
    chosen = random.choices(feature_types, k=num_features)
    data = {}
    for obj in [1, 2]:
        for i, t in enumerate(chosen, start=1):
            data[f"Obj{obj}_Feat{i}"] = generate_feature(t, num_samples)

    labels = []
    for idx in range(num_samples):
        matches = sum(
            data[f"Obj1_Feat{i}"][idx] == data[f"Obj2_Feat{i}"][idx]
            for i in range(1, num_features+1)
        )
        labels.append(1 if matches >= num_features//2 else 0)
    df = pd.DataFrame(data)
    y = np.array(labels)
    return df, y


def create_substrate(input_dim):
    input_coords = [(-1.0 + 2.0 * i / (input_dim - 1), -1.0)
                    for i in range(input_dim)]
    output_coords = [(0.0, 1.0)]
    return Substrate(input_coordinates=input_coords,
                     output_coordinates=output_coords,
                     hidden_coordinates=[])


def run_es_hyperneat(X_train, y_train, X_test, y_test, gens=30):
    input_dim = X_train.shape[1]
    substrate = create_substrate(input_dim)
    params = DefaultESParameters().__dict__

    config_path = 'config-feedforward'
    config = neat.Config(
        neat.DefaultGenome,
        neat.DefaultReproduction,
        neat.DefaultSpeciesSet,
        neat.DefaultStagnation,
        config_path
    )

    def eval_genomes(genomes, config_obj):
        for gid, genome in genomes:
            cppn = create_cppn(genome, config_obj)
            es_net = ESNetwork(substrate, cppn, params)
            pheno = es_net.create_phenotype_network()
            preds = [int(pheno.activate(x)[0] > 0.5) for x in X_train]
            genome.fitness = f1_score(y_train, preds, average='macro')

    pop = neat.Population(config)
    pop.add_reporter(neat.StdOutReporter(True))
    pop.add_reporter(neat.StatisticsReporter())
    winner = pop.run(eval_genomes, gens)

    cppn_w = create_cppn(winner, config)
    es_net_w = ESNetwork(substrate, cppn_w, params)
    pheno_w = es_net_w.create_phenotype_network()
    preds_test = [int(pheno_w.activate(x)[0] > 0.5) for x in X_test]
    f1 = f1_score(y_test, preds_test, average='macro')
    print(f"F1-score ES-HyperNEAT: {f1:.4f}")

    os.makedirs('saved_models', exist_ok=True)
    with open(f'saved_models/es_hyperneat_winner.pkl', 'wb') as f:
        pickle.dump(winner, f)

    return winner


if __name__ == '__main__':
    X_s, y_s = create_dataset(num_features=4, num_samples=30)
    X_s = preprocess_dataset(X_s)
    X_s = X_s.values.astype(float)
    X_ts, X_vs, y_ts, y_vs = train_test_split(
        X_s, y_s, test_size=0.2, random_state=42)
    run_es_hyperneat(X_ts, y_ts, X_vs, y_vs, gens=30)

    X_b, y_b = create_dataset(num_features=15, num_samples=1500)
    X_b = preprocess_dataset(X_b)
    X_b = X_b.values.astype(float)
    X_tb, X_vb, y_tb, y_vb = train_test_split(
        X_b, y_b, test_size=0.2, random_state=42)
    run_es_hyperneat(X_tb, y_tb, X_vb, y_vb, gens=50)


 ****** Running generation 0 ****** 

Population's average fitness: 0.40801 stdev: 0.05486
Best fitness: 0.42857 - size: (1, 5) - species 1 - id 2
Average adjusted fitness: 0.148
Mean genetic distance 1.253, standard deviation 0.386
Population of 50 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0    50      0.4    0.148     0
Total extinctions: 0
Generation time: 3.084 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.40516 stdev: 0.06353
Best fitness: 0.55556 - size: (1, 5) - species 1 - id 61
Average adjusted fitness: 0.146
Mean genetic distance 1.027, standard deviation 0.351
Population of 50 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1    50      0.6    0.146     0
Total extinctions: 0
Generation time: 2.162 sec (2.623 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.41741 stdev: 0.047